In [51]:
import pandas as pd 
import numpy as np
from datetime import datetime as dt 
import matplotlib.pyplot as plt
%matplotlib inline
from darts import TimeSeries

In [44]:
tfl_df = pd.read_csv(r'C:\Users\steph\OneDrive\Documents\Data\TFL\tfl-journeys-type.csv', parse_dates=['Period beginning'])
# Convert num cols
num_cols = ['Reporting Period', 'Days in period', 'Bus journeys (m)', 'Underground journeys (m)', 'DLR Journeys (m)', 'Tram Journeys (m)', 'Overground Journeys (m)']
tfl_df[num_cols] = tfl_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)
tfl_df['Reporting Period'] = tfl_df['Reporting Period'].astype(int)
# Drop any periods as 13
tfl_df = tfl_df[tfl_df['Reporting Period']<13]
# Create better date column
tfl_df['new_period'] = pd.to_datetime(dict(year=tfl_df['Period beginning'].dt.year, month=tfl_df['Reporting Period'], day=1))
# Drop ccolumns not required
tfl_df = tfl_df.drop(columns=['Period and Financial year', 'Reporting Period', 'Period beginning','Period ending', 'Emirates Airline Journeys (m)', 'TfL Rail Journeys (m)'])
# Remove nan (before overground) and reset index
tfl_df = tfl_df.dropna().reset_index(drop=True)
tfl_df#.info()

,Days in period,Bus journeys (m),Underground journeys (m),DLR Journeys (m),Tram Journeys (m),Overground Journeys (m),new_period
0,28.0,179.9,90.3,6.7,2.2,5.6,2010-08-01
1,28.0,178.8,90.6,6.4,2.3,5.4,2010-09-01
2,28.0,140.1,72.5,4.8,1.8,3.5,2010-10-01
3,28.0,183.0,84.4,6.3,2.1,5.2,2011-11-01
4,28.0,177.2,87.8,6.5,2.2,5.2,2011-12-01
...,...,...,...,...,...,...,...
130,28.0,115.2,58.2,5.9,1.2,8.8,2021-06-01
131,28.0,129.8,69.7,7.1,1.7,10.6,2021-07-01
132,28.0,125.0,73.7,7.2,1.7,11.5,2021-08-01
133,28.0,130.9,75.8,7.0,1.6,12.2,2021-09-01


In [45]:
tfl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Days in period            135 non-null    float64       
 1   Bus journeys (m)          135 non-null    float64       
 2   Underground journeys (m)  135 non-null    float64       
 3   DLR Journeys (m)          135 non-null    float64       
 4   Tram Journeys (m)         135 non-null    float64       
 5   Overground Journeys (m)   135 non-null    float64       
 6   new_period                135 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(6)
memory usage: 7.5 KB


In [47]:
tfl_df.describe(include='all').T

C:\Users\steph\anaconda3\envs\montVenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Days in period,135.0,NaN,NaT,NaN,NaT,NaT,28.022222,0.448325,26.0,28.0,28.0,28.0,31.0
Bus journeys (m),135.0,NaN,NaT,NaN,NaT,NaT,161.126667,37.21013,30.2,155.2,175.2,183.25,203.4
Underground journeys (m),135.0,NaN,NaT,NaN,NaT,NaT,89.878519,26.009159,5.7,88.25,97.5,105.85,118.2
DLR Journeys (m),135.0,NaN,NaT,NaN,NaT,NaT,7.74,2.026732,1.2,6.9,8.2,9.3,10.6
Tram Journeys (m),135.0,NaN,NaT,NaN,NaT,NaT,2.05037,0.466629,0.4,2.0,2.2,2.3,2.6
Overground Journeys (m),135.0,NaN,NaT,NaN,NaT,NaT,11.081481,3.678258,1.0,8.7,10.9,14.4,17.8
new_period,135,135,2019-06-01,1,2010-08-01,2021-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
tube_df = tfl_df[['new_period', 'Underground journeys (m)']].rename(columns={'Underground journeys (m)':'journeys'})
tube_series = TimeSeries.from_dataframe(tube_df, time_col='new_period', freq='m')
#tube_df

[2022-03-16 11:56:14,225] ERROR | darts.timeseries | ValueError: Could not correctly fill missing dates with the observed/passed frequency freq='m'. Not all input time stamps contained in the newly created TimeSeries. For more information about frequency aliases, read https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases


ValueError: Could not correctly fill missing dates with the observed/passed frequency freq='m'. Not all input time stamps contained in the newly created TimeSeries. For more information about frequency aliases, read https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases